In [ ]:
class Hand:
    def __init__(self):
        self.cards = []
        self.evaluate()
        
    
    def evaluate(self):
        self.value = sum([card.value for card in self.cards])
        self.aces = [card.rank for card in self.cards].count('A')
        while self.value > 21 and self.aces > 0:
            self.value -= 10
            self.aces -= 1
    
    def add_card(self, card):
        self.cards.append(card)
        self.evaluate()
        return self.value
    
    def is_blackjack(self):
        return self.value == 21 and len(self.cards) == 2

    def __repr__(self):
        return f'<Hand: {self.value} {[str(card) for card in self.cards]}>'
    
    

In [ ]:
# HOUSE RULES
SPLITTING_SAME_VALUE = True # if you can split same-valued but different-ranked cards


In [ ]:
class Game:
    def __init__(self, d, number_decks=6):
        self.deck = Deck(number_decks)
        self.deck.shuffle()
        self.d = d
        self.d.hit_button.on_click(self.hit)
        self.d.stand_button.on_click(self.stand)
        print(f'New game with {self.deck}')

    def dealer_draw_card(self):
        card = self.deck.deal()
        print(f"Dealer draws {card}")
        self.dealer.add_card(card)
        self.d.dealer_cards_label.value = str(self.dealer)
    
    def player_draw_card(self):
        card = self.deck.deal()
        print(f"Player draws {card}")
        self.player.add_card(card)
        self.d.player_cards_label.value = str(self.player)

    def player_turn(self):
        self.player_draw_card()
        self.player_draw_card()
        time.sleep(0.1)

        if self.player.is_blackjack():
            print("Player has a Blackjack!")

        elif self.player.cards[0].rank == self.player.cards[1].rank:
            self.d.split_button.disabled = False
            # TODO: implement splitting

        # TODO: implement doubling down
        
        if self.player.value < 21:
            self.d.hit_button.disabled = False
            self.d.stand_button.disabled = False

    def hit(self, b):
        with self.d.output:
            self.player_draw_card()
            if self.player.value > 21:
                print("Player busts!")
                self.d.hit_button.disabled = True
                self.d.stand_button.disabled = True
                self.finish_round()

            elif self.player.value == 21:
                print("Player stands at 21!")
                self.d.hit_button.disabled = True
                self.d.stand_button.disabled = True
                self.finish_round()

    def stand(self, b):
        with self.d.output:
            print("Player stands at", self.player.value)
            self.finish_round()
           
    def finish_round(self):
        self.dealer_turn()
        self.payout = self.calculate_payout(self.player)
        self.d.disable_action_buttons()
        self.d.deal_button.disabled = False
    
    def dealer_turn(self):
        while self.dealer.value < 17:
            self.dealer_draw_card()
            time.sleep(0.1)
            if self.dealer.value > 21:
                break
            elif self.dealer.value >= 17:
                print("Dealer stands at", self.dealer.value)
                break

    def calculate_payout(self, player_hand):
        if player_hand.value > 21:
            print("Player busts!")
            return -1
        if self.dealer.value > 21:
            print("Dealer busts!")
            return 1

        if player_hand.is_blackjack():
            if self.dealer.is_blackjack():
                print("Player and Dealer have a Blackjack!")
                print("Push!")
                return 0
            else: 
                print("Player wins with a Blackjack!")
                return 1.5
        
        if self.dealer.value == player_hand.value:
            print("Push!")
            return 0
        
        if self.dealer.value > player_hand.value:
            print("Dealer wins!")
            return -1
        
        if player_hand.value > self.dealer.value:
            print("Player wins!")
            return 1

    def play_round(self):
        # setup
        self.player = Hand()
        self.dealer = Hand()

        # dealer first card
        self.dealer_draw_card()
        
        # player
        self.player_turn()
